# [Clusterization tesk](https://www.kaggle.com/competitions/clustering-physical-activity-data/)

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from sklearn.cluster import KMeans
from modules import fulling_indexes, normalize_cluster_labels
from pipeline import GroupRobustScaler, process_data
import seaborn as sns

In [2]:
df = pd.read_parquet("data.parquet")

In [3]:
import numpy as np
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
imputer.fit_transform(df)

array([[ 2.52750e+02,  3.48750e+01,  9.02847e-01, ...,  7.63201e-01,
        -5.03652e-01,  5.00000e+00],
       [ 3.65245e+03,  2.97500e+01, -4.06933e+00, ...,  4.05888e-01,
        -4.23791e-01,  2.00000e+00],
       [ 5.04760e+02,  3.43750e+01, -2.00754e+00, ...,  3.93796e-01,
         6.06316e-01,  8.00000e+00],
       ...,
       [ 2.47984e+03,  3.46875e+01, -6.36755e+00, ...,  7.56767e-01,
         1.51899e-01,  5.00000e+00],
       [ 3.43782e+03,  2.80625e+01,  5.46194e+00, ..., -5.66677e-02,
        -5.95815e-01,  6.00000e+00],
       [ 8.11750e+02,  3.33125e+01, -9.22591e+00, ...,  2.27375e-01,
        -5.81235e-01,  4.00000e+00]])

In [4]:
df.to_parquet("data_KNN_10.parquet")

In [4]:
# Обработка данных и экспорта
pipeline, pca_data, labels = process_data(df)

Processing pipeline: 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]


In [5]:
# Получаем индексы из очищенных данных
clean_df = df.dropna(subset=df.columns.difference(['subject_id', 'timestamp']))
filtered_index = clean_df.index

In [6]:
# Создаем результирующий DataFrame только с валидными индексами
result_df = pd.DataFrame({
    'Index': filtered_index,
    'activityID': normalize_cluster_labels(labels, filtered_index)
})

# Заполнение NaN мусором.
result_df = fulling_indexes(result_df)

# Экспорт результатов
result_df.to_csv("activity_clusters.csv", index=False)

In [ ]:
# Оптимизация кластеров с обработкой ошибок
def optimize_clusters(data):
    scores = []
    for k in tqdm(range(2, 20), desc='Optimizing clusters'):
        kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
        labels = kmeans.fit_predict(data)
        scores.append(silhouette_score(data, labels))
    return scores


sns.scatterplot(optimize_clusters(pd.DataFrame(pca_data).sample(1000)))

Optimizing clusters:   0%|          | 0/18 [00:12<?, ?it/s]


KeyboardInterrupt: 